# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [2]:
STOCKS_DATA_PATH = './sp_500_stocks.csv'

stocks_list = pd.read_csv(STOCKS_DATA_PATH)
stocks_list.count()

Ticker    505
dtype: int64

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [4]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'

data = requests.get(api_url).json()
print(data)

{'avgTotalVolume': 96521944, 'calculationPrice': 'close', 'change': 0.92, 'changePercent': 0.0068, 'close': 138.08, 'closeSource': 'alficfio', 'closeTime': 1696661955112, 'companyName': 'Apple Inc', 'currency': 'USD', 'delayedPrice': 134.96, 'delayedPriceTime': 1729108466547, 'extendedChange': 1.06, 'extendedChangePercent': 0.00808, 'extendedPrice': 134.75, 'extendedPriceTime': 1678514712532, 'high': 138.08, 'highSource': '1d   entlpamiiyduc5reee', 'highTime': 1728478898459, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 139.07, 'iexCloseTime': 1711850653677, 'iexLastUpdated': 1664606952646, 'iexMarketPercent': 0.01873604707493016, 'iexOpen': 139.15, 'iexOpenTime': 1676048894904, 'iexRealtimePrice': 133.6, 'iexRealtimeSize': 104, 'iexVolume': 1550433, 'lastTradeTime': 1705576851673, 'latestPrice': 133.37, 'latestSource': 'Close', 'latestTime': 'June 14, 2022', 'latestUpdate': 1685901695441, 'latestVolume': 87790534, 'low': 133.76, 'lowSource': 'lr Xe 

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [5]:
price = data['latestPrice']
marketcap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [6]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
df = pd.DataFrame(columns=my_columns)
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [7]:
appending_df = pd.DataFrame([[symbol, price, marketcap, 'N/A']], columns=my_columns)
appending_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,133.37,2155585671680,N/A


In [8]:
df = pd.concat([df, appending_df])
df = df.drop([0])
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [9]:
for stock in stocks_list['Ticker'][:5]:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    curr_data = requests.get(api_url).json()

    symbol = curr_data['symbol']
    price = curr_data['latestPrice']
    marketcap = curr_data['marketCap']
    appending_df = pd.DataFrame([[symbol, price, marketcap, 'N/A']], columns=my_columns)

    df = pd.concat([df, appending_df], ignore_index=True)

In [10]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,120.99,35458044446,N/A
1,AAL,13.24,8768553460,N/A
2,AAP,176.37,10439780553,N/A
3,AAPL,137.48,2169774346542,N/A
4,ABBV,137.72,247298368328,N/A


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [11]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [12]:
symbol_groups = list(chunks(stocks_list['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_df = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    batch_data = requests.get(batch_api_url).json()
    
    for symbol in symbol_string.split(','):
        if (symbol in batch_data):
            price = batch_data[symbol]['quote']['latestPrice']
            marketcap = batch_data[symbol]['quote']['marketCap']

            appending_df = pd.DataFrame([[symbol, price, marketcap, 'N/A']], columns=my_columns)
            final_df = pd.concat([final_df, appending_df], ignore_index=True)
        else:
            print('Missiing Symbol:', symbol)

final_df

Missiing Symbol: DISCA
Missiing Symbol: HFC
Missiing Symbol: VIAC
Missiing Symbol: WLTW


,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,118.24,35659437027,N/A
1,AAL,13.05,8618311928,N/A
2,AAP,178.12,10721284502,N/A
3,AAPL,135.76,2248575035248,N/A
4,ABBV,138.24,253819620228,N/A
...,...,...,...,...
496,YUM,113.23,31757797816,N/A
497,ZBH,105.33,22858262684,N/A
498,ZBRA,312.13,16223092483,N/A
499,ZION,52.64,8212780915,N/A


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [13]:
portfolio_size = input('Enter the value of your portfolio:')

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number!\nPlease enter an integer...")
    portfolio_size = input('Enter the value of your portfolio:')
    val = float(portfolio_size)

print('Your Portfolio size is:', portfolio_size)

Your Portfolio size is: 1000000


In [14]:
position_size = val/final_df.shape[0]

for i in range (0, len(final_df.index)):
    final_df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_df.loc[i, 'Stock Price'])

final_df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,118.24,35659437027,16
1,AAL,13.05,8618311928,152
2,AAP,178.12,10721284502,11
3,AAPL,135.76,2248575035248,14
4,ABBV,138.24,253819620228,14
...,...,...,...,...
496,YUM,113.23,31757797816,17
497,ZBH,105.33,22858262684,18
498,ZBRA,312.13,16223092483,6
499,ZION,52.64,8212780915,37


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [15]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_df.to_excel(writer, 'Recommended Trades', index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [16]:
bg_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1,
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1,
    }
)

integer_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': bg_color,
        'border': 1
    }
)

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [17]:
# writer.sheets['Recommended Trades'].set_column('A:A', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 18, string_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 18, string_format)
# writer.save()

# writer.sheets['Recommended Trades'].write('A1', my_columns[0], string_format)
# writer.sheets['Recommended Trades'].write('B1', my_columns[1], string_format)
# writer.sheets['Recommended Trades'].write('C1', my_columns[2], string_format)
# writer.sheets['Recommended Trades'].write('D1', my_columns[3], string_format)

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [18]:
column_formats = {
    'A': [my_columns[0], string_format],
    'B': [my_columns[1], dollar_format],
    'C': [my_columns[2], dollar_format],
    'D': [my_columns[3], integer_format]
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [19]:
writer.save()